# TensorFlow!

In [ ]:
import tensorflow
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

client1_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client2_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client3_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')

# It's training time!

In [ ]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = numpy.load('dataset.npz') # Download dataset.npz from the link in Readme.md
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v

x_train, y_train = get_dataset()

In [ ]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

client1_x, client2_x, client1_y, client2y = train_test_split(x_train, y_train, test_size=0.33, random_state=100)
len(client1_x)
len(client2_x)

In [ ]:
# Dellocate variable to save RAM space
del x_train
del y_train

In [ ]:
client1_x, client3_x, client1_y, client3y = train_test_split(
    client1_x, client1_y, test_size=0.5, random_state=100)
print(len(client1_x), len(client2_x), len(client3_x))


In [ ]:
"""
Training client 1 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = '/checkpoint/f2-1/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client1_model.fit(client1_x, client1_y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


In [ ]:
"""
Training client 2 model
"""
checkpoint_filepath = '/checkpoint/f2-2/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client2_model.fit(client2_x, client2y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


In [ ]:
"""
Training client 3 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = '/checkpoint/f2-3/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client3_model.fit(client3_x, client3y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


In [ ]:
"""
Aggregrate models
"""

client1_weights = client1_model.get_weights()
client2_weights = client2_model.get_weights()
client3_weights = client3_model.get_weights()

client1_weights = 1/3.35 * client1_weights
client2_weights = 1/3.3 * client2_weights
client3_weights = 1/3.35 * client3_weights

aggregated_weights = numpy.add(client1_weights,client2_weights)
aggregated_weights = numpy.add(aggregated_weights, client3_weights)

client1_weights.set_weights(aggregated_weights)
client1_weights.save_weights("model_weight/federated2_weights", save_format="h5")
